In [1]:
import requests
import requests_cache
import urllib.parse
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
import concurrent.futures
import time
import threading
import hashlib
import os

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}

proxies = {
    'http': 'http://10.10.113.97:7890',
    'https': 'http://10.10.113.97:7890'
}

In [4]:
session = requests_cache.CachedSession('.cache')

爬取书本详情

In [166]:
data = []
error_pages = []
keyword = 'medicine'
for page in tqdm(range(1, 101)):
    try:
        url = f"https://libgen.is/search.php?&res=100&req={keyword}&phrase=1&view=detailed&column=def&sort=def&sortmode=ASC&page={page}"
        res = session.get(url=url, headers=headers, proxies=proxies, timeout=30)
        assert res.status_code == 200
        soup = BeautifulSoup(res.text, 'lxml')
        tables = [t for t in soup.body.find_all('table', border="0", rules="cols", width="100%") if t.col is not None]
        data += [get_books_detail(table) for table in tables]
    except Exception as _:
        error_pages.append(page)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:08<00:00,  1.88s/it]


In [2]:
# df = pd.DataFrame(data).set_index('id')
# df.to_csv(f'textbooks-details.csv')
df = pd.read_csv('textbooks-details.csv').set_index('id')

In [3]:
df

,md5,title,author,series,publisher,year,edition,language,pages,isbn,size,extension,detail_link
id,,,,,,,,,,,,,
13,D13DCCAC55E0F9B17C9D7AC267A96D4F,Mathematical models in biology. An introduction,"Elizabeth S. Allman, John A. Rhodes",NaN,Cambridge University Press,2003,1,English,385,"9780521525862, 0521525861",1 Mb (1341232),pdf,http://library.lol/main/D13DCCAC55E0F9B17C9D7A...
14,59F98A1FFBFF47448335DA76C11B7F51,Mathematical models in biology: solution manual,"Elizabeth S. Allman, John A. Rhodes",NaN,Cambridge University Press,2003,1,English,81,"9780521525862, 0521525861",451 Kb (461441),pdf,http://library.lol/main/59F98A1FFBFF47448335DA...
16,C60F9DB2CC956AF8B59B4DBBCB311B32,The organic codes: an introduction to semantic...,Marcello Barbieri,NaN,Cambridge University Press,2002,NaN,English,316,"0521531004, 9780521531009, 0521824141, 9780521...",2 Mb (2304145),pdf,http://library.lol/main/C60F9DB2CC956AF8B59B4D...
29,EE801E1CA0B5E588A7D20EC158DF15EA,"Cell biology, a short course","Stephen R. Bolsover, Jeremy S. Hyams, Elizabet...",NaN,Wiley-Liss,2003,2,English,535,"0471263931, 9780471263937, 9780471461593",7 Mb (7031589),djvu,http://library.lol/main/EE801E1CA0B5E588A7D20E...
37,95BFBEFF10BE691C67A018CA41FB3E2C,Computational molecular biology: an introduction,"Peter Clote, Rolf Backofen",Wiley series in mathematical and computational...,Wiley,2000,1,English,307,"9780471872511, 0-471-87251-2, 0471872520",3 Mb (3527027),djvu,http://library.lol/main/95BFBEFF10BE691C67A018...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2247345,DD3E43D061CF7799A0B49A6D0C0FAAFE,Stem Cell Therapy: A Rising Tide: How Stem Cel...,Neil H. Riordan,NaN,self-publ.,2017,1st,English,0[341],"099904530X, 9780999045305",4 Mb (4100509),pdf,http://library.lol/main/DD3E43D061CF7799A0B49A...
2248051,FE6D11AE5A14A5E9D476026012156E26,Revolutions and Continuity in Greek Mathematics,"Sialaros, Michalis (ed.)","Science, Technology and Medicine in Ancient Cu...",De Gruyter,2018,NaN,English,391[404],"9783110563658, 3110563657, 9783110565959, 9783...",3 Mb (3529683),pdf,http://library.lol/main/FE6D11AE5A14A5E9D47602...
2248617,345DE87F77A25AFC5FCD4212E65AF3F4,Are You Afraid of Snakes?: A Doctor’s Explorat...,C. Scott Mahan,NaN,MedMaster Inc.,2017,1,English,257,"1935660268, 9781935660262",349 Kb (357445),epub,http://library.lol/main/345DE87F77A25AFC5FCD42...


爬取书本文件链接

In [12]:
def get_books_pdf_link(textbook: pd.Series):
    try:
        detail_link = textbook['detail_link']
        res = session.get(url=detail_link, headers=headers, proxies=proxies, timeout=30)
        assert res.status_code == 200
        soup = BeautifulSoup(res.text, 'lxml')
        link = soup.body.table.find('a', string='GET').get('href')
        return (textbook.name, link)
    except Exception as _:
        return (textbook.name, '')

In [14]:
get_books_pdf_link(df.iloc[9600])

(1077732,
 'https://download.library.lol/main/1077000/4fdd07b94cbaf8e32f3a724f8cf338cf/%28Contemporary%20Biomedicine%203%29%20James%20E.%20Wheeler%20%28auth.%29%2C%20Ivan%20Damjanov%2C%20Barbara%20B.%20Knowles%2C%20Davor%20Solter%20%28eds.%29%20-%20The%20Human%20Teratomas_%20Experimental%20and%20Clinical%20Biology-Humana%20Press%20%281983%29.pdf')

In [15]:
thread_pool_size = 5

In [16]:
with concurrent.futures.ThreadPoolExecutor(max_workers=thread_pool_size) as executor:
    results = executor.map(get_books_pdf_link, [row for _, row in df.iterrows()])
links = [{'id': res[0], 'link': res[1]} for res in results]
ldf = pd.DataFrame(links).set_index('id').dropna()
print('finished')

finished


In [39]:
pub_nums = df.value_counts('publisher').to_dict()
pub_names = [pub for pub, num in pub_nums.items() if num > 100]
print(pub_names)

['Humana Press', 'Springer US', 'Springer-Verlag Berlin Heidelberg', 'Springer Netherlands', 'Springer-Verlag New York', 'Springer', 'Academic Press', 'Cambridge University Press', 'CRC Press', 'Springer International Publishing', 'Wiley-Blackwell', 'Saunders', 'Oxford University Press, USA', 'Academic Press,  Elsevier', 'Elsevier, Academic Press', 'National Academies Press', 'Elsevier Science', 'Oxford University Press', 'Lippincott Williams & Wilkins', 'Wiley', 'Informa Healthcare', 'Elsevier', 'Springer-Verlag London', 'Routledge']


In [43]:
exclude_words = [
    'algorithm', 'analysis', 'analyze', 'analytical', 'biophysic', 'biomines', 'chemistry', 'climate', 'computer', 'construction', 'construct', 'cryptology', 'cryptography', 'digital', 'economic', 'electro', 'geometry', 'industry', 'internet', 'physic', 'mathematic', 'material', 'religion'
]

In [4]:
# related_textbooks = df[(df['extension'] == 'pdf') & (df['language'] == 'English') & (df['publisher'].isin(pub_names)) & (~df['title'].apply(lambda x: any(exclude_word in str(x).lower() for exclude_word in exclude_words)))]
related_textbooks = pd.read_csv('related_textbooks.csv').set_index('id')
related_textbooks

,md5,title,author,series,publisher,year,edition,language,pages,isbn,size,extension,detail_link,link
id,,,,,,,,,,,,,,
16,C60F9DB2CC956AF8B59B4DBBCB311B32,The organic codes: an introduction to semantic...,Marcello Barbieri,NaN,Cambridge University Press,2002,NaN,English,316,"0521531004, 9780521531009, 0521824141, 9780521...",2 Mb (2304145),pdf,http://library.lol/main/C60F9DB2CC956AF8B59B4D...,https://download.library.lol/main/0/c60f9db2cc...
122,C2242AB0CDD631EEAC1DD6C31F6492D0,Ribosomes,A. S Spirin,"Molecular biology, biochemistry and biophysics",Springer-Verlag New York,1969,NaN,English,337,NaN,4 Mb (4555116),pdf,http://library.lol/main/C2242AB0CDD631EEAC1DD6...,https://download.library.lol/main/0/c2242ab0cd...
880343,DD386E1BD85F2FA983109DA49E107B64,Human Molecular Biology: An Introduction to th...,Richard J. Epstein,NaN,Cambridge University Press,2002,1,English,656[655],"052164285X, 9780521642859",209 Mb (218858648),pdf,http://library.lol/main/DD386E1BD85F2FA983109D...,https://download.library.lol/main/880000/dd386...
880327,DB99C64ED073D4160ED44A061D421914,Marrow Stromal Cell Culture,"Jon N. Beresford, Maureen E. Owen",Handbooks in Practical Animal Cell Biology,Cambridge University Press,1998,1,English,165,"0521580218, 9780521580212",16 Mb (16887836),pdf,http://library.lol/main/DB99C64ED073D4160ED44A...,https://download.library.lol/main/880000/db99c...
22126,948C3D4D1CF014DD5F776D7B48BEB86D,"HPLC of Peptides and Proteins, Methods and Pro...",Marie-Isabel Aguilar,Methods in Molecular Biology,Humana Press,2003,1,English,395,"9780896039773, 0896039773",3 Mb (3372661),pdf,http://library.lol/main/948C3D4D1CF014DD5F776D...,https://download.library.lol/main/22000/948c3d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244218,A1B6F94C714A376544D7B01D5AA59607,55th years German Society of Anaesthesiology a...,"Schüttler, Jürgen",NaN,Springer,2012,NaN,English,312 S.[326],"9783540242574, 3540242570",16 Mb (16327351),pdf,http://library.lol/main/A1B6F94C714A376544D7B0...,https://download.library.lol/main/2244000/a1b6...
2245075,3AB8C4681D4A302DB95F0A70E7D9F474,Emergency Medicine Secrets,"Vincent J. Markovchick, Peter T. Pons, Katheri...",NaN,Elsevier,2016,6,English,906,NaN,11 Mb (11828394),pdf,http://library.lol/main/3AB8C4681D4A302DB95F0A...,https://download.library.lol/main/2245000/3ab8...
2245798,27DB546FC7CA6BE5F2CCBB55ACD976BD,"Bioethics, Medicine and the Criminal Law Volum...",Amel Alghrani; Rebecca Bennett; Suzanne Ost (e...,Cambridge Bioethics and Law,Cambridge University Press,2013,NaN,English,307,"1139789694, 9781139789691",3 Mb (3629901),pdf,http://library.lol/main/27DB546FC7CA6BE5F2CCBB...,https://download.library.lol/main/2245000/27db...


In [55]:
for index, textbook in tqdm(related_textbooks.iterrows(), total=len(related_textbooks)):
    if not textbook['link'].isna():
        continue
    link = get_books_pdf_link(textbook)[1]
    if link != '':
        related_textbooks.at[index, 'link'] = link

100%|████████████████████████████████████████████████████████████████████████████| 10959/10959 [29:41<00:00,  6.15it/s]


In [6]:
len(related_textbooks[~related_textbooks['link'].isna()])

10949

In [97]:
def checksum_file(file):
    md5 = hashlib.md5()
    with open(file, 'rb') as f:
        while True:
            chunk = f.read(4096)
            if not chunk:
                break
            md5.update(chunk)
    return md5.hexdigest()

In [99]:
def download_textbook_pdf(link, filename, filemd5='', save_dir=os.path.join(os.curdir, 'download')):
    try:
        if filemd5 != '' and filename in os.listdir(save_dir):
            if checksum_file(os.path.join(save_dir, filename)).lower() == filemd5.lower():
                return
        res = requests.get(link, stream=True)
        assert res.status_code == 200
        with open(os.path.join(save_dir, filename), 'wb') as f:
            for chunk in res.iter_content(1024):
                f.write(chunk)
        if filemd5 != '':
            assert checksum_file(os.path.join(save_dir, filename)).lower() == filemd5.lower()
        return os.path.join(save_dir, filename)
    except Exception as e:
        print(e)
        return ''

In [14]:
download_links = related_textbooks[related_textbooks['size'].apply(lambda x: 20*1024*1024 < int(x.split('(')[-1].replace(')', '')) < 30*1024*1024)]['link'].to_list()

In [15]:
len(download_links)

872

In [16]:
with open('links-10000-10872.txt', 'w', encoding='utf-8') as f:
    for link in download_links[0:873]:
        f.write(str(link) + '\n')

In [ ]:
import pandas as pd
import numpy as np
import concurrent.futures
import subprocess

related_textbooks = pd.read_csv('related_textbooks.csv')
related_textbooks = related_textbooks.set_index('id')
to_download_textbooks = related_textbooks[related_textbooks['size'].apply(lambda x: 1024*1024 < int(x.split('(')[-1].replace(')', '')) < 10*1024*1024)]
urls = to_download_textbooks['link'].to_list()
md5s = to_download_textbooks['md5'].to_list()


def download_file(url, md5):
    subprocess.run(["wget", "-q", "-c", "-O", f'{md5}.pdf', url])

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(download_file, url, md5) for url, md5 in zip(urls, md5s)]
    for future in concurrent.futures.as_completed(futures):
        future.result()

In [116]:
related_textbooks[related_textbooks['size'].apply(lambda x: int(x.split('(')[-1].replace(')', '')) < 10*1024*1024)]

,md5,title,author,series,publisher,year,edition,language,pages,isbn,size,extension,detail_link,link
id,,,,,,,,,,,,,,
16,C60F9DB2CC956AF8B59B4DBBCB311B32,The organic codes: an introduction to semantic...,Marcello Barbieri,NaN,Cambridge University Press,2002,NaN,English,316,"0521531004, 9780521531009, 0521824141, 9780521...",2 Mb (2304145),pdf,http://library.lol/main/C60F9DB2CC956AF8B59B4D...,https://download.library.lol/main/0/c60f9db2cc...
122,C2242AB0CDD631EEAC1DD6C31F6492D0,Ribosomes,A. S Spirin,"Molecular biology, biochemistry and biophysics",Springer-Verlag New York,1969,NaN,English,337,NaN,4 Mb (4555116),pdf,http://library.lol/main/C2242AB0CDD631EEAC1DD6...,https://download.library.lol/main/0/c2242ab0cd...
22126,948C3D4D1CF014DD5F776D7B48BEB86D,"HPLC of Peptides and Proteins, Methods and Pro...",Marie-Isabel Aguilar,Methods in Molecular Biology,Humana Press,2003,1,English,395,"9780896039773, 0896039773",3 Mb (3372661),pdf,http://library.lol/main/948C3D4D1CF014DD5F776D...,https://download.library.lol/main/22000/948c3d...
22135,89A0FF0B3E9FC3CA6AC48B1A5D345750,Adipose Tissue Protocols,Gérard Ailhaud,Methods in Molecular Biology,Humana Press,2001,1,English,350[317],"9780896037472, 9781592592319, 0896037479",3 Mb (3577219),pdf,http://library.lol/main/89A0FF0B3E9FC3CA6AC48B...,https://download.library.lol/main/22000/89a0ff...
22286,1F6DB3C0A0F6298A02DB4948D901DDF3,Oxidative Stress Biomarkers and Antioxidant Pr...,"Maria Giulia Battelli, Silvia Musiani (auth.),...",Methods in Molecular Biology 186,Humana Press,2002,1,English,322[319],"9780896038509, 0896038505",4 Mb (3699593),pdf,http://library.lol/main/1F6DB3C0A0F6298A02DB49...,https://download.library.lol/main/22000/1f6db3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232365,A53A75459F43FBC59CB206644E2781ED,The Identity of the History of Science and Med...,Andrew Cunningham,Variorum Collected Studies 1003,Routledge,2012,1,English,284[223],"1409440249, 9781409440246",5 Mb (5297017),pdf,http://library.lol/main/A53A75459F43FBC59CB206...,https://download.library.lol/main/2232000/a53a...
2232446,9F66AD94E6282D3AF02A3AA6D34C87EC,Patient-Centered Medicine: Transforming the Cl...,Moira Stewart et al.,Patient-Centered Care Series,CRC Press,2013,3,English,442[492],"1846195667, 9781846195662",3 Mb (2813415),pdf,http://library.lol/main/9F66AD94E6282D3AF02A3A...,https://download.library.lol/main/2232000/9f66...
2233303,7EA07863A359D1A93E461424EF67E21F,Methodologies for the Rhetoric of Health and M...,"Lisa Meloncon, J. Blake Scott (eds.)",NaN,Routledge,2018,NaN,English,330,"1315303744, 9781315303741",4 Mb (3719392),pdf,http://library.lol/main/7EA07863A359D1A93E4614...,https://download.library.lol/main/2233000/7ea0...


In [102]:
test_book = related_textbooks.iloc[1]
test_book['link']

'https://download.library.lol/main/0/c2242ab0cdd631eeac1dd6c31f6492d0/%28Molecular%20biology%2C%20biochemistry%20and%20biophysics%29%20A.%20S%20Spirin%20-%20Ribosomes-Springer-Verlag%20New%20York%20%281969%29.pdf'

In [104]:
download_textbook_pdf(test_book['link'], f"{test_book['md5']}.{test_book['extension']}", test_book['md5'])

'.\\download\\C2242AB0CDD631EEAC1DD6C31F6492D0.pdf'

In [5]:
def get_books_detail(table: BeautifulSoup):
    try:
        tbody = table.tbody
        image = tbody.find('td', string="Title").find_previous_sibling('td').a.img.get('src', '')
        pre = image.split('/')[-2]
        title_td = tbody.find('td', string="Title").find_next_sibling('td')
        title = title_td.text
        author_td = tbody.find('td', string="Author(s):").find_next_sibling('td')
        author = author_td.text
        series_td = tbody.find('td', string="Series:").find_next_sibling('td')
        series = series_td.text
        publisher_td = tbody.find('td', string="Publisher:").find_next_sibling('td')
        publisher = publisher_td.text
        year_td = tbody.find('td', string="Year:").find_next_sibling('td')
        year = year_td.text
        edition_td = tbody.find('td', string="Edition:").find_next_sibling('td')
        edition = edition_td.text
        language_td = tbody.find('td', string="Language:").find_next_sibling('td')
        language = language_td.text
        pages_td = tbody.find('td', string="Pages:").find_next_sibling('td')
        pages = pages_td.text
        isbn_td = tbody.find('td', string="ISBN:").find_next_sibling('td')
        isbn = isbn_td.text
        id_td = tbody.find('td', string="ID:").find_next_sibling('td')
        id_ = id_td.text
        size_td = tbody.find('td', string="Size:").find_next_sibling('td')
        size = size_td.text
        extension_td = tbody.find('td', string="Extension:").find_next_sibling('td')
        extension = extension_td.text
        md5 = title_td.b.a.get('href').split('md5=')[-1]
        detail_link = f"http://library.lol/main/{md5}"
        file_link = f"https://download.library.lol/main/{pre}/{md5.lower()}/{md5.upper()}.{extension.lower()}"
        return {
            'id': id_,
            'md5': md5,
            'title': title,
            'author': author,
            'series': series,
            'publisher': publisher,
            'year': year,
            'edition': edition,
            'language': language,
            'pages': pages,
            'isbn': isbn,
            'size': size,
            'extension': extension,
            'detail_link': detail_link,
            'file_link': file_link
        }
    except Exception as _:
        return {}